In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import *
import tensorflow as tf
from solo.getdata import *
import tushare as ts

%matplotlib inline
%load_ext autoreload
%autoreload 2

/Users/mahui/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/mahui/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Users/mahui/anaconda/lib/python2.7/site-packages/sklearn/lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)


# 一、获取数据
## 参数说明：
- 'name'：名称
- 'start'：开始时间
- 'data_type'：数据处理方式，'normal'：原始数据, 'carg'：复合增长率, 'grow'：增长速度
- 'ktype' ：时间周期

In [2]:
new_data = data_recieve(name='000837',start = '2008-10-01',ktype='D',data_type = 'carg')

起始时间：	 2008-10-06
结束时间：	 2017-03-07
数据个数：	 1966

      open  close   high    low    volume
180  6.356  6.288  6.405  6.092  25489.71
181  6.112  6.356  6.434  6.063  13945.27
182  6.366  6.307  6.366  6.209  20080.54
183  6.405  6.258  6.434  6.209  15268.85
184  5.965  5.711  6.131  5.711  36454.38

查看最新数据：

                open      high     close       low    volume
2008-10-07 -0.038389  0.004528  0.010814 -0.004760 -0.452906
2008-10-08  0.041558 -0.010569 -0.007709  0.024080  0.439953
2008-10-09  0.006126  0.010682 -0.007769  0.000000 -0.239620
2008-10-10 -0.068696 -0.047094 -0.087408 -0.080206  1.387500
2008-10-13 -0.104946 -0.030338  0.030818 -0.065137 -0.313828




# 二、建立特征
## 参数说明：
- 'data'：数据集
- 'time'：多个时间周期
- 'normal'：是否对数据进行归一化
- 'pattern' ：是否加入k模式识别

In [3]:
ml_datas = data_indicator(data=new_data,time=[5,10,20,30,60],normal=True)

In [4]:
#dataf = ts.get_k_data('399300',start = '2004-10-01',ktype='D')
#new_data.to_csv('new.csv')
#ml_datas.to_csv('ml_data.csv')
#dataf.to_csv('data.csv')


# 三、设定X和Y

In [5]:
num_labels = 10 #设定要分成多少个类

ml_datas['Price'] = pd.qcut(ml_datas['target'],num_labels, precision=1) #将target进行10等分

In [6]:
#y设定为符合增长率求整
X_ori = ml_datas.drop(['target','Price'],axis=1).values
y_ori = ml_datas['Price'].values

y_ori = pd.get_dummies(y_ori)
y_columns = y_ori.columns.values
y_ori = y_ori.values


print('X形状：')
print(X_ori.shape)
print('y形状：')
print(y_ori.shape)

X形状：
(1610, 784)
y形状：
(1610, 10)


In [7]:
print('Price分割：')
for i in y_columns:
    print(i)

Price分割：
[-10, -3.5]
(-3.5, -2.1]
(-2.1, -1.1]
(-1.1, -0.4]
(-0.4, 0.1]
(0.1, 0.6]
(0.6, 1.2]
(1.2, 2.1]
(2.1, 3.4]
(3.4, 10.1]


# 四、转换格式
## 将原2维数组(N,D)转换成（N,D,H,W）的格式

In [8]:
image_size = 28
num_channels = 10 #多少天的数据（即多少个通道）

X_1,y = data_trans(X_ori,y_ori,days=num_channels,depth=image_size)

X = np.transpose(X_1,(0,3,2,1))

X.shape,y.shape

((1601, 28, 28, 10), (1601, 10))

# 五、按8:1:1的比例分割Train、Test和Valid

In [9]:
train_dataset_1, test_dataset, train_labels_1, test_labels = cross_validation.train_test_split(X, y, test_size=0.1, random_state=77)
train_dataset, valid_dataset, train_labels, valid_labels = cross_validation.train_test_split(train_dataset_1, train_labels_1, test_size=0.1, random_state=77)

In [10]:
#转成float32，不转似乎不行


def reformat(dataset, labels):
    dataset = dataset.reshape(
        (-1, image_size, image_size, num_channels)).astype(np.float32)

    labels = labels.astype(np.float32)
    #labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
    return dataset, labels


train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

('Training set', (1296, 28, 28, 10), (1296, 10))
('Validation set', (144, 28, 28, 10), (144, 10))
('Test set', (161, 28, 28, 10), (161, 10))


In [11]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

# 六、Tensorflow



## 1、cnn

In [18]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal([patch_size*patch_size*depth*depth//25, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)    
        hidden = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME') 
        shape = hidden.get_shape().as_list()    
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases) 

        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    
    l2_loss1 = tf.nn.l2_loss(layer1_weights)
    l2_loss2 = tf.nn.l2_loss(layer2_weights)
    l2_loss3 = tf.nn.l2_loss(layer3_weights)
    l2_loss4 = tf.nn.l2_loss(layer4_weights)




    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + 0.00001 * (l2_loss1+l2_loss2+l2_loss3+l2_loss4)
    
    # Optimizer.
    optimizer = tf.train.GradientDescentOptimizer(5e-5).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))






num_steps = 1001

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run(
          [optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (step % 50 == 0):
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
  
    cc = np.array(valid_prediction.eval())

Initialized
Minibatch loss at step 0: 3.488656
Minibatch accuracy: 25.0%
Validation accuracy: 13.9%
Minibatch loss at step 50: 3.413373
Minibatch accuracy: 12.5%
Validation accuracy: 11.1%
Minibatch loss at step 100: 4.074187
Minibatch accuracy: 6.2%
Validation accuracy: 11.1%
Minibatch loss at step 150: 3.217750
Minibatch accuracy: 6.2%
Validation accuracy: 9.7%
Minibatch loss at step 200: 3.041208
Minibatch accuracy: 12.5%
Validation accuracy: 11.1%
Minibatch loss at step 250: 2.956689
Minibatch accuracy: 6.2%
Validation accuracy: 13.2%
Minibatch loss at step 300: 2.783059
Minibatch accuracy: 0.0%
Validation accuracy: 11.8%
Minibatch loss at step 350: 2.784541
Minibatch accuracy: 0.0%
Validation accuracy: 10.4%
Minibatch loss at step 400: 2.347641
Minibatch accuracy: 18.8%
Validation accuracy: 11.8%
Minibatch loss at step 450: 2.537210
Minibatch accuracy: 6.2%
Validation accuracy: 11.1%
Minibatch loss at step 500: 2.747462
Minibatch accuracy: 6.2%
Validation accuracy: 9.7%
Minibatch 


## 2、CrossValidation



In [25]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

    # Input data.
    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    # Variables.
    layer1_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, num_channels, depth], stddev=0.1))
    layer1_biases = tf.Variable(tf.zeros([depth]))
    
    layer2_weights = tf.Variable(tf.truncated_normal([patch_size, patch_size, depth, depth], stddev=0.1))
    layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
    
    layer3_weights = tf.Variable(tf.truncated_normal([patch_size*patch_size*depth*depth//25, num_hidden], stddev=0.1))
    layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
    
    layer4_weights = tf.Variable(tf.truncated_normal([num_hidden, num_labels], stddev=0.1))
    layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
    # Model.
    def model(data):
        conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer1_biases)
        conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
        hidden = tf.nn.relu(conv + layer2_biases)    
        hidden = tf.nn.max_pool(hidden, [1,2,2,1], [1,2,2,1], padding='SAME') 
        shape = hidden.get_shape().as_list()    
        reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
        hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases) 

        return tf.matmul(hidden, layer4_weights) + layer4_biases
  
    # Training computation.
    logits = model(tf_train_dataset)
    
    l2_loss1 = tf.nn.l2_loss(layer1_weights)
    l2_loss2 = tf.nn.l2_loss(layer2_weights)
    l2_loss3 = tf.nn.l2_loss(layer3_weights)
    l2_loss4 = tf.nn.l2_loss(layer4_weights)


    aa = [1e-6,5e-6,1e-5,5e-5,1e-4,5e-4,1e-3,5e-3,1e-2,5e-2,1e-1,5e-1]
    bb = [1e-6,5e-6,1e-5,5e-5,1e-4,5e-4,1e-3,5e-3,1e-2,5e-2,1e-1,5e-1]
    for a in aa:
        for b in bb:
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + 0.00001 * (l2_loss1+l2_loss2+l2_loss3+l2_loss4)
    
            # Optimizer.
            optimizer = tf.train.GradientDescentOptimizer(5e-5).minimize(loss)
  
            # Predictions for the training, validation, and test data.
            train_prediction = tf.nn.softmax(logits)
            valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
            test_prediction = tf.nn.softmax(model(tf_test_dataset))
            
            num_steps = 20001
            with tf.Session(graph=graph) as session:
                tf.global_variables_initializer().run()
            
                for step in range(num_steps):
                    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
                    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
                    batch_labels = train_labels[offset:(offset + batch_size), :]
                    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
                    _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)

            
                print('Test accuracy: %.1f%%,\t%%' % accuracy(test_prediction.eval(), test_labels),a,b)
  

('Test accuracy: 12.4%,\t%', 1e-06, 1e-06)
('Test accuracy: 12.4%,\t%', 1e-06, 5e-06)
('Test accuracy: 11.8%,\t%', 1e-06, 1e-05)
('Test accuracy: 13.7%,\t%', 1e-06, 5e-05)
('Test accuracy: 12.4%,\t%', 1e-06, 0.0001)
('Test accuracy: 9.3%,\t%', 1e-06, 0.0005)
('Test accuracy: 13.0%,\t%', 1e-06, 0.001)
('Test accuracy: 11.8%,\t%', 1e-06, 0.005)
('Test accuracy: 12.4%,\t%', 1e-06, 0.01)
('Test accuracy: 13.7%,\t%', 1e-06, 0.05)
('Test accuracy: 10.6%,\t%', 1e-06, 0.1)
('Test accuracy: 16.1%,\t%', 1e-06, 0.5)
('Test accuracy: 12.4%,\t%', 5e-06, 1e-06)
('Test accuracy: 9.9%,\t%', 5e-06, 5e-06)
('Test accuracy: 13.7%,\t%', 5e-06, 1e-05)
('Test accuracy: 9.9%,\t%', 5e-06, 5e-05)
('Test accuracy: 8.7%,\t%', 5e-06, 0.0001)
('Test accuracy: 7.5%,\t%', 5e-06, 0.0005)
('Test accuracy: 9.3%,\t%', 5e-06, 0.001)
('Test accuracy: 10.6%,\t%', 5e-06, 0.005)
('Test accuracy: 13.0%,\t%', 5e-06, 0.01)
('Test accuracy: 13.0%,\t%', 5e-06, 0.05)
('Test accuracy: 9.9%,\t%', 5e-06, 0.1)
('Test accuracy: 9.9%,\t

('Test accuracy: 12.4%,\t%', 1e-06, 1e-06)
('Test accuracy: 12.4%,\t%', 1e-06, 5e-06)
('Test accuracy: 11.8%,\t%', 1e-06, 1e-05)
('Test accuracy: 13.7%,\t%', 1e-06, 5e-05)
('Test accuracy: 12.4%,\t%', 1e-06, 0.0001)
('Test accuracy: 9.3%,\t%', 1e-06, 0.0005)
('Test accuracy: 13.0%,\t%', 1e-06, 0.001)
('Test accuracy: 11.8%,\t%', 1e-06, 0.005)
('Test accuracy: 12.4%,\t%', 1e-06, 0.01)
('Test accuracy: 13.7%,\t%', 1e-06, 0.05)
('Test accuracy: 10.6%,\t%', 1e-06, 0.1)
('Test accuracy: 16.1%,\t%', 1e-06, 0.5)
('Test accuracy: 12.4%,\t%', 5e-06, 1e-06)
('Test accuracy: 9.9%,\t%', 5e-06, 5e-06)
('Test accuracy: 13.7%,\t%', 5e-06, 1e-05)
('Test accuracy: 9.9%,\t%', 5e-06, 5e-05)
('Test accuracy: 8.7%,\t%', 5e-06, 0.0001)
('Test accuracy: 7.5%,\t%', 5e-06, 0.0005)
('Test accuracy: 9.3%,\t%', 5e-06, 0.001)
('Test accuracy: 10.6%,\t%', 5e-06, 0.005)
('Test accuracy: 13.0%,\t%', 5e-06, 0.01)
('Test accuracy: 13.0%,\t%', 5e-06, 0.05)
('Test accuracy: 9.9%,\t%', 5e-06, 0.1)
('Test accuracy: 9.9%,\t%', 5e-06, 0.5)
('Test accuracy: 11.8%,\t%', 1e-05, 1e-06)
('Test accuracy: 12.4%,\t%', 1e-05, 5e-06)
('Test accuracy: 8.1%,\t%', 1e-05, 1e-05)
('Test accuracy: 10.6%,\t%', 1e-05, 5e-05)
('Test accuracy: 10.6%,\t%', 1e-05, 0.0001)
('Test accuracy: 10.6%,\t%', 1e-05, 0.0005)
('Test accuracy: 15.5%,\t%', 1e-05, 0.001)
('Test accuracy: 13.7%,\t%', 1e-05, 0.005)
('Test accuracy: 13.7%,\t%', 1e-05, 0.01)
('Test accuracy: 12.4%,\t%', 1e-05, 0.05)
('Test accuracy: 11.2%,\t%', 1e-05, 0.1)
('Test accuracy: 11.8%,\t%', 1e-05, 0.5)
('Test accuracy: 9.9%,\t%', 5e-05, 1e-06)
('Test accuracy: 10.6%,\t%', 5e-05, 5e-06)
('Test accuracy: 11.2%,\t%', 5e-05, 1e-05)
('Test accuracy: 12.4%,\t%', 5e-05, 5e-05)
('Test accuracy: 11.2%,\t%', 5e-05, 0.0001)
('Test accuracy: 13.7%,\t%', 5e-05, 0.0005)
('Test accuracy: 13.0%,\t%', 5e-05, 0.001)
('Test accuracy: 10.6%,\t%', 5e-05, 0.005)
('Test accuracy: 16.8%,\t%', 5e-05, 0.01)
('Test accuracy: 8.7%,\t%', 5e-05, 0.05)
('Test accuracy: 11.8%,\t%', 5e-05, 0.1)
('Test accuracy: 10.6%,\t%', 5e-05, 0.5)
('Test accuracy: 13.7%,\t%', 0.0001, 1e-06)
('Test accuracy: 14.9%,\t%', 0.0001, 5e-06)
('Test accuracy: 10.6%,\t%', 0.0001, 1e-05)
('Test accuracy: 13.7%,\t%', 0.0001, 5e-05)
('Test accuracy: 11.2%,\t%', 0.0001, 0.0001)
('Test accuracy: 7.5%,\t%', 0.0001, 0.0005)
('Test accuracy: 8.1%,\t%', 0.0001, 0.001)
('Test accuracy: 8.7%,\t%', 0.0001, 0.005)
('Test accuracy: 7.5%,\t%', 0.0001, 0.01)
('Test accuracy: 11.8%,\t%', 0.0001, 0.05)
('Test accuracy: 11.2%,\t%', 0.0001, 0.1)
('Test accuracy: 14.9%,\t%', 0.0001, 0.5)
('Test accuracy: 10.6%,\t%', 0.0005, 1e-06)
('Test accuracy: 16.8%,\t%', 0.0005, 5e-06)
('Test accuracy: 14.9%,\t%', 0.0005, 1e-05)
('Test accuracy: 16.1%,\t%', 0.0005, 5e-05)
('Test accuracy: 14.3%,\t%', 0.0005, 0.0001)
('Test accuracy: 9.9%,\t%', 0.0005, 0.0005)
('Test accuracy: 10.6%,\t%', 0.0005, 0.001)
('Test accuracy: 8.7%,\t%', 0.0005, 0.005)
('Test accuracy: 10.6%,\t%', 0.0005, 0.01)
('Test accuracy: 10.6%,\t%', 0.0005, 0.05)
('Test accuracy: 12.4%,\t%', 0.0005, 0.1)
('Test accuracy: 13.0%,\t%', 0.0005, 0.5)
('Test accuracy: 11.8%,\t%', 0.001, 1e-06)
('Test accuracy: 6.8%,\t%', 0.001, 5e-06)
('Test accuracy: 13.0%,\t%', 0.001, 1e-05)
('Test accuracy: 9.9%,\t%', 0.001, 5e-05)
('Test accuracy: 13.0%,\t%', 0.001, 0.0001)
('Test accuracy: 12.4%,\t%', 0.001, 0.0005)
('Test accuracy: 9.3%,\t%', 0.001, 0.001)
('Test accuracy: 11.8%,\t%', 0.001, 0.005)
('Test accuracy: 11.8%,\t%', 0.001, 0.01)
('Test accuracy: 11.2%,\t%', 0.001, 0.05)
('Test accuracy: 10.6%,\t%', 0.001, 0.1)
('Test accuracy: 14.3%,\t%', 0.001, 0.5)
('Test accuracy: 13.7%,\t%', 0.005, 1e-06)
('Test accuracy: 6.2%,\t%', 0.005, 5e-06)
('Test accuracy: 8.1%,\t%', 0.005, 1e-05)
('Test accuracy: 10.6%,\t%', 0.005, 5e-05)
('Test accuracy: 11.2%,\t%', 0.005, 0.0001)
('Test accuracy: 11.2%,\t%', 0.005, 0.0005)
('Test accuracy: 12.4%,\t%', 0.005, 0.001)
('Test accuracy: 12.4%,\t%', 0.005, 0.005)
('Test accuracy: 8.7%,\t%', 0.005, 0.01)
('Test accuracy: 8.1%,\t%', 0.005, 0.05)
('Test accuracy: 10.6%,\t%', 0.005, 0.1)
('Test accuracy: 11.2%,\t%', 0.005, 0.5)
('Test accuracy: 7.5%,\t%', 0.01, 1e-06)
('Test accuracy: 10.6%,\t%', 0.01, 5e-06)
('Test accuracy: 17.4%,\t%', 0.01, 1e-05)
('Test accuracy: 14.3%,\t%', 0.01, 5e-05)
('Test accuracy: 8.7%,\t%', 0.01, 0.0001)
('Test accuracy: 15.5%,\t%', 0.01, 0.0005)
('Test accuracy: 9.3%,\t%', 0.01, 0.001)
('Test accuracy: 8.1%,\t%', 0.01, 0.005)
('Test accuracy: 11.2%,\t%', 0.01, 0.01)
('Test accuracy: 11.2%,\t%', 0.01, 0.05)
('Test accuracy: 9.3%,\t%', 0.01, 0.1)
('Test accuracy: 8.1%,\t%', 0.01, 0.5)
('Test accuracy: 10.6%,\t%', 0.05, 1e-06)
('Test accuracy: 9.9%,\t%', 0.05, 5e-06)
('Test accuracy: 12.4%,\t%', 0.05, 1e-05)
('Test accuracy: 9.9%,\t%', 0.05, 5e-05)
('Test accuracy: 13.7%,\t%', 0.05, 0.0001)
('Test accuracy: 11.8%,\t%', 0.05, 0.0005)
('Test accuracy: 9.9%,\t%', 0.05, 0.001)
('Test accuracy: 14.9%,\t%', 0.05, 0.005)
('Test accuracy: 9.3%,\t%', 0.05, 0.01)
('Test accuracy: 14.3%,\t%', 0.05, 0.05)
('Test accuracy: 8.7%,\t%', 0.05, 0.1)
('Test accuracy: 7.5%,\t%', 0.05, 0.5)
('Test accuracy: 10.6%,\t%', 0.1, 1e-06)
('Test accuracy: 9.9%,\t%', 0.1, 5e-06)
('Test accuracy: 9.9%,\t%', 0.1, 1e-05)
('Test accuracy: 13.0%,\t%', 0.1, 5e-05)
('Test accuracy: 11.8%,\t%', 0.1, 0.0001)
('Test accuracy: 7.5%,\t%', 0.1, 0.0005)
('Test accuracy: 7.5%,\t%', 0.1, 0.001)
('Test accuracy: 13.0%,\t%', 0.1, 0.005)
('Test accuracy: 11.2%,\t%', 0.1, 0.01)
('Test accuracy: 10.6%,\t%', 0.1, 0.05)
('Test accuracy: 11.2%,\t%', 0.1, 0.1)
('Test accuracy: 10.6%,\t%', 0.1, 0.5)
('Test accuracy: 8.1%,\t%', 0.5, 1e-06)
('Test accuracy: 13.0%,\t%', 0.5, 5e-06)
('Test accuracy: 10.6%,\t%', 0.5, 1e-05)
('Test accuracy: 10.6%,\t%', 0.5, 5e-05)
('Test accuracy: 11.2%,\t%', 0.5, 0.0001)
('Test accuracy: 10.6%,\t%', 0.5, 0.0005)
('Test accuracy: 9.9%,\t%', 0.5, 0.001)
('Test accuracy: 11.2%,\t%', 0.5, 0.005)
('Test accuracy: 12.4%,\t%', 0.5, 0.01)
('Test accuracy: 6.8%,\t%', 0.5, 0.05)
('Test accuracy: 11.8%,\t%', 0.5, 0.1)
('Test accuracy: 7.5%,\t%', 0.5, 0.5)